In [20]:
import pandas as pd
import os

In [ ]:
file_path = r"C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies"
print(file_path)

C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies


In [ ]:
list_of_files = os.listdir(file_path)
path_dir = []
for path in list_of_files:
    new_path = f"{file_path}\\{path}"
    path_dir.append(new_path)


['Ahluwalia Contracts India Ltd', 'Ajmera Realty and Infra India Ltd', 'Alpine Housing Dev Corp Ltd', 'AMJ Land Ltd', 'Ansal Housing Ltd', 'Arihant Foundations and Housing Ltd', 'Arihant Superstructures Ltd', 'Arvind SmartSpaces Ltd', 'Ashiana Housing Ltd', 'Brigade Enterprises Ltd', 'Deepak Builders & Engineers India Ltd', 'Dilip Buildcon Ltd', 'Equinox India Developments Ltd', 'Espire Hospitality Ltd', 'Ganesh Housing Corporation Ltd', 'Godrej Properties Ltd', 'Hazoor Multi Projects Ltd', 'Hubtown Ltd', 'Kesar India Ltd', 'Keystone Realtors Ltd', 'Kolte-Patil Developers Ltd', 'Lancor Holdings Ltd', 'Landmark Property Development Company Ltd', 'Macrotech Developers Ltd', 'Mahindra Lifespace Developers Ltd', 'Marathon Nextgen Realty Ltd', 'Navkar Urbanstructure Ltd', 'NBCC (India) Ltd', 'Oberoi Realty Ltd', 'Omaxe Ltd', 'Parsvnath Developers Ltd', 'Peninsula Land Ltd', 'Phoenix Mills Ltd', 'Prajay Engineers Syndicate Ltd', 'Prerna Infrabuild Ltd', 'Prestige Estates Projects Ltd', 'Prim

In [48]:
import os
import pandas as pd
import json

for file_dir in path_dir:
    company_list_dir = os.listdir(file_dir)
    
    for company in company_list_dir:
        company_path = os.path.join(file_dir, company)
        
        # Skip if it's not a directory
        if not os.path.isdir(company_path):
            continue  

        new_file_path = os.path.join(company_path, "Pruned_Excel")
        
        # Skip if "Pruned_Excel" folder doesn't exist
        if not os.path.exists(new_file_path):
            print(f"⚠️ Skipping: {new_file_path} (Not Found)")
            continue

        # Initialize dictionary for storing parameters
        company_params = {
            "Balance-sheet": [],
            "Cash-flow": [],
            "Profit-loss": [],
            "Quarterly-resul": [],
            "Ratios": [],
            "Year": []
        }

        # Get list of Excel files
        prunedExcel_file_list = [
            file for file in os.listdir(new_file_path)
            if file.endswith(".xlsx") or file.endswith(".xls")
        ]
        
        for excel_file in prunedExcel_file_list:
            excel_file_path = os.path.join(new_file_path, excel_file)
            df = pd.read_excel(excel_file_path)

            parameters = df.columns[1:].tolist()  # Extract column names excluding the first column

            if "Balance-sheet" in excel_file:
                company_params["Balance-sheet"].extend(parameters)
            elif "Cash-flow" in excel_file:
                company_params["Cash-flow"].extend(parameters)
            elif "Profit-loss" in excel_file:
                company_params["Profit-loss"].extend(parameters)
            elif "Quarterly-resul" in excel_file:
                company_params["Quarterly-resul"].extend(parameters)
            elif "Ratios" in excel_file:
                company_params["Ratios"].extend(parameters)
            elif "Year" in excel_file:
                company_params["Year"].extend(parameters)
        
        # Save the JSON file inside the respective company directory
        json_file_path = os.path.join(company_path, "company_params.json")
        with open(json_file_path, "w") as f:
            json.dump(company_params, f, indent=4)

        print(f"✅ JSON file saved at: {json_file_path}")


✅ JSON file saved at: C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\Construction - Residential & Commercial Complexes\Ahluwalia Contracts India Ltd\company_params.json
✅ JSON file saved at: C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\Construction - Residential & Commercial Complexes\Ajmera Realty and Infra India Ltd\company_params.json
✅ JSON file saved at: C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\Construction - Residential & Commercial Complexes\Alpine Housing Dev Corp Ltd\company_params.json
✅ JSON file saved at: C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\Construction - Residential & Commercial Complexes\AMJ Land Ltd\company_params.json
✅ JSON file saved at: C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tu

In [29]:
for file_dir in path_dir:  # Iterate over each company sector folder
    file_path = os.path.join(file_dir, "company_data.txt")  # Full path to the file

    if os.path.exists(file_path):  # Check if the file exists
        os.remove(file_path)  # Delete the file
        print(f"Deleted: {file_path}")  # Log deletion
    else:
        print(f"Not found: {file_path}")

Deleted: C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\Construction - Residential & Commercial Complexes\company_data.txt
Deleted: C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\Hospital & Healthcare Services\company_data.txt
Deleted: C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\IT Services & Consulting\company_data.txt
Deleted: C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\Oil Exploration and Production\company_data.txt
Deleted: C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\Transport Infrastructure\company_data.txt


In [32]:
import os

# Iterate through each company sector (folder)
for file_dir in path_dir:  
    company_list_dir = os.listdir(file_dir)
    sector_common_params = {}

    # Ensure a directory exists to store common parameters
    common_params_folder = os.path.join(file_dir, "Common_Parameters")
    os.makedirs(common_params_folder, exist_ok=True)
    output_file = os.path.join(common_params_folder, f"{os.path.basename(file_dir)}_Common_Parameters.txt")

    for company in company_list_dir:  
        txt_file_path = os.path.join(file_dir, f"{company}.txt")
        if not os.path.exists(txt_file_path):
            continue  

        with open(txt_file_path, "r") as f:
            lines = f.readlines()

        current_category = None  
        company_params = {}

        for line in lines:
            line = line.strip()

            # Detect category headers (e.g., "Balance Sheet:")
            if line.endswith(":") and " " in line:
                current_category = line[:-1]  
                company_params[current_category] = set()
            elif current_category and line:  
                company_params[current_category].add(line)

        # Merge this company's parameters into sector_common_params
        for category, params in company_params.items():
            if category not in sector_common_params:
                sector_common_params[category] = params  # First company's parameters
            else:
                sector_common_params[category] &= params  # Keep only common parameters

    # Save the final filtered parameters
    with open(output_file, "w") as f:
        for category, params in sector_common_params.items():
            f.write(f"{category}:\n")
            for param in sorted(params):  
                f.write(f"    {param}\n")
            f.write("\n")

    print(f"Saved common parameters for {os.path.basename(file_dir)} in {output_file}")


Saved common parameters for Construction - Residential & Commercial Complexes in C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\Construction - Residential & Commercial Complexes\Common_Parameters\Construction - Residential & Commercial Complexes_Common_Parameters.txt
Saved common parameters for Hospital & Healthcare Services in C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\Hospital & Healthcare Services\Common_Parameters\Hospital & Healthcare Services_Common_Parameters.txt
Saved common parameters for IT Services & Consulting in C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\IT Services & Consulting\Common_Parameters\IT Services & Consulting_Common_Parameters.txt
Saved common parameters for Oil Exploration and Production in C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Finan